## Improving Quantum Circuit performance with TKET on H-Series devices and other platforms

#### Estimating Two-Qubit Gater Error

$ \% \text{Error}_{2QG} = (1-\text{Fidelity}^N) \times 100\% $

where $N$ i the number of two qubit gates in system.

Examples:

$100$ 2QGs with the fidelity of $0.993$, which means the error% of $=(1-0.993^{100}) \times 100\% = 50\%$

$100$ 2QGs with the fidelity of $0.998$, which means the error% of $=(1-0.998^{100}) \times 100\% = 18\%$

$100$ 2QGs with the fidelity of $0.998$, which means the error% of $=(1-0.999^{100}) \times 100\% = 9.5\%$



![alt text](img.png "2 Qubit Gate Fidelity")

## TKET Extensions

![alt text](img2.png "Title")

## TKET has a default pass manager for each backend

`get_compiled_circuit(circuit,optimization_level)`

![alt text](img3.png "Title")

In [2]:
from pytket.extensions.quantinuum import QuantinuumBackend

machine = 'H1-1'

backend = QuantinuumBackend(device_name = machine)
backend.login()

In [3]:
print(machine, "status: ", QuantinuumBackend.device_state(device_name = machine))

H1-1 status:  online


In [4]:
[x.device_name for x in QuantinuumBackend.available_devices()]

['H1-1SC', 'H1-1E', 'H1-1']

Take a look [here](https://github.com/CQCL/pytket/tree/main/examples)